# Budget-based SVR

The basic SVR algorithm solves the following optimization problem:

$$\min \frac{1}{2} ||w||^2 + C \sum(\xi_i + \hat\xi_i)$$

under the constraints

$$y_i - \epsilon - \xi_i \leq w \cdot x_i + b
          \leq y_i + \epsilon + \hat\xi_i \\
  \xi_i, \hat\xi_i \geq 0$$

The Wolfe dual is

$$\min_{\alpha, \hat\alpha} \frac{1}{2} \sum_{i, j}
  (\alpha_i - \hat\alpha_i) (\alpha_j - \hat\alpha_j) k(x_i, x_j)
  - \sum_i (\alpha_i - \hat\alpha_i) y_i
  + \sum_i (\alpha_i + \hat\alpha_i) \epsilon$$

under the constraints

$$\sum_i (\alpha_i - \hat\alpha_i) = 0 \\
  0 \leq \alpha_i, \hat\alpha_i \leq C$$

The budget-based version, instead, focuses on

$$\min_{\alpha, \hat\alpha, \gamma} \frac{1}{2} \sum_{i, j}
  (\alpha_i - \hat\alpha_i) (\alpha_j - \hat\alpha_j) k(x_i, x_j)
  - \sum_i (\alpha_i - \hat\alpha_i) y_i
  + \sum_i (\alpha_i + \hat\alpha_i) \epsilon
  + \gamma B$$

under the constraints

$$\sum_i (\alpha_i - \hat\alpha_i) = 0 \\
  \alpha_i - \gamma \leq C  \\
  \hat\alpha_i - \gamma \leq C \\
  \alpha_i, \hat\alpha_i, \gamma \geq 0$$

The KKT conditions are

$$\alpha_i(w \cdot x_i + b - y_i + \epsilon + \xi_i) = 0 \\
  \hat\alpha_i(y_i + \epsilon + \hat\xi_i - w \cdot x_i - b) = 0 \\
  \beta_i \xi_i = 0, \hat\beta_i \hat\xi_i = 0 \\
  \gamma B \sum(\xi_i + \hat\xi_i) = 0$$

So that if the optimal value of $\gamma$ is zero, $b$ is
found as usual, otherwise $b = y_i - \epsilon - w \cdot x_i$ with $i$ such
that $\alpha_i < C + \gamma$, still considering optimal values of
variables. Note that in both cases $b$ can be found considering $i$ such
that $\alpha_i < C + \gamma$. Similar considerations hold for the hatted
set of variables.

In [22]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from optimization import GurobiSolver
from kernel import LinearKernel, GaussianKernel

def generate_data():
    n = 50
    a = 1.5
    b = 0.3

    X = np.random.random(n)
    y = a * X + b + np.random.random(n) / 2

    return X, y

def feature_dot(x_new, sv, kernel):
    return sum([a * kernel.compute(x, x_new) for (a, x) in sv])

X, y = generate_data()
s = GurobiSolver()


In [23]:

@widgets.interact(C=widgets.FloatSlider(min=1E-3, max=1E3, step=.001, value=1),
                  sigma=widgets.FloatSlider(min=1E-4, max=3, step=0.1, 
                                           value=0.1),
                  epsilon=widgets.FloatSlider(min=0.01, max=1,
                                              step=.01, value=.1),
                  B=widgets.FloatSlider(min=0.01, max=10, step=.5, value=1)
                  )
def f(C, sigma, epsilon, B):
    k = GaussianKernel(sigma)
    alpha, alpha_hat = s.solve(X, y, C=1, kernel=k, epsilon=epsilon)

    sv = [(a - a_h, x) for a, a_h, x in zip(alpha, alpha_hat, X)
                                     if a - a_h != 0]
    
    global aaa
    aaa = sv

    b_values = [y_ - feature_dot(x, sv, k) - epsilon
                for a, x, y_ in zip(alpha, X, y) if 0 < a < C]
    b_values += [y_ - feature_dot(x, sv, k) + epsilon
                for a_h, x, y_ in zip(alpha_hat, X, y) if 0 < a_h < C]

    b = np.mean(b_values)

    regression = lambda x: feature_dot(x, sv, k) + b

    x_values = np.linspace(min(X), max(X), 500)

    fig, ax = plt.subplots()
    ax.scatter(X, y)
    ax.plot(x_values, [regression(x) for x in x_values])
    ax.plot(x_values, [regression(x) + epsilon for x in x_values],
            'k--', linewidth=.5)
    ax.plot(x_values, [regression(x) - epsilon for x in x_values],
            'k--', linewidth=.5)

    alpha, alpha_hat, gamma = s.solve(X, y, C, k, epsilon, budget=B)

    budget_sv = [(a - a_h, x) for a, a_h, x
             in zip(alpha, alpha_hat, X) if a - a_h != 0]


    budget_b_values = [y_ - feature_dot(x, sv, k) - epsilon
                for a, x, y_ in zip(alpha, X, y) if 0 < a < C + gamma]
    budget_b_values += [y_ - feature_dot(x, sv, k) + epsilon
                for a_h, x, y_ in zip(alpha_hat, X, y) if 0 < a_h < C + gamma]

    budget_b = np.mean(b_values)

    budget_regression = lambda x: feature_dot(x, budget_sv, k) + budget_b

    ax.plot(x_values, [budget_regression(x) + epsilon for x in x_values],
            'r:', linewidth=.5)
    ax.plot(x_values, [budget_regression(x) for x in x_values], 'r')
    ax.plot(x_values, [budget_regression(x) - epsilon for x in x_values],
            'r:', linewidth=.5)
    #return alpha, alpha_hat

interactive(children=(FloatSlider(value=1.0, description='C', max=1000.0, min=0.001, step=0.001), FloatSlider(…

In [20]:
from IPython.display import HTML

aaa

[(-0.14607637805692372, 0.08193296324325083),
 (1.0, 0.5918910492252069),
 (0.8367673983653987, 0.9900179076520604),
 (-1.0, 0.7576616026504494),
 (0.5541807459418351, 0.9108530533878771),
 (-1.0, 0.6897674522377063),
 (1.0, 0.8126005189000531),
 (1.0, 0.8017943113965676),
 (1.0, 0.5902669031437341),
 (-1.0, 0.7456327043424169),
 (0.3651893925107828, 0.19788269389906565),
 (-0.6229912710641062, 0.8580934472955833),
 (-0.417628936474119, 0.5771851809735215),
 (-0.1544629104012109, 0.6873149476315689),
 (1.0, 0.9403299431495786),
 (-1.0, 0.5836830412550431),
 (1.0, 0.7020549958199833),
 (-1.0, 0.8783658986695058),
 (-1.0, 0.025882049443798127),
 (-1.0, 0.13910490213658167),
 (0.18416131633797603, 0.3404673263247383),
 (-1.0, 0.31980677049811557),
 (1.0, 0.021209808391162155),
 (-1.0, 0.9729121658576204),
 (-1.0, 0.028317754364300174),
 (0.8835249927611366, 0.45081540674292875),
 (1.0, 0.7219266343368003),
 (0.5173356541942926, 0.281950385407295),
 (1.0, 0.07077790049623378),
 (-1.0, 0.50